In [9]:
# !pip install -U --user /kaggle/input/lama-nn/LightAutoML-0.3.7.4-py3-none-any.whl >> none

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import copy
import gc
import os
import time
import lightautoml
import bisect

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# target = pd.read_csv("tr.csv", usecols=['user_id', 'is_male', 'age'])
# # test = pd.read_csv("/kaggle/input/mlcupmlfeaturesv6/test_ml_num_v2.csv", nrows=10000)
# # test.sort_values("user_id", inplace=True)

# print(len(target))

In [4]:
# oof_age_1 = pd.read_csv("oof_warm_age_lin_lgbm_automl.csv")
# oof_age_2 = pd.read_csv("oof_warm_age_big_automl.csv")

# test_age_1 = pd.read_csv("test_warm_age_lin_lgbm_automl.csv")
# test_age_2 = pd.read_csv("test_warm_age_big_automl.csv")

# pca_train = pd.read_csv("pca_train.csv")
# pca_test = pd.read_csv("pca_test.csv")

# fl_train = pd.read_csv("fl_train_11.csv")
# fl_test = pd.read_csv("fl_test_11.csv")

# train_age_dart_pred = pd.read_csv("train_age_dart_pred.csv")
# test_age_dart_pred = pd.read_csv("test_age_dart_pred.csv")

# age_gru_oof = pd.read_csv("age_gru_oof.csv")
# age_gru_test = pd.read_csv("age_gru_test.csv")

In [5]:
# oof_age = oof_age_1.merge(oof_age_2, on='user_id')
# test_age = test_age_1.merge(test_age_2, on='user_id')

# oof_age = oof_age.merge(pca_train, on='user_id')
# test_age = test_age.merge(pca_test, on='user_id')

# oof_age = oof_age.merge(fl_train, on='user_id')
# test_age = test_age.merge(fl_test, on='user_id')

# oof_age = oof_age.merge(train_age_dart_pred, on='user_id', how ='left')
# test_age = test_age.merge(test_age_dart_pred, on='user_id', how ='left')

# oof_age = oof_age.merge(age_gru_oof, on='user_id', how ='left')
# test_age = test_age.merge(age_gru_test, on='user_id', how ='left')

# oof_age.head()

In [6]:
# oof_age = oof_age.merge(target, on='user_id')
# print(len(oof_age))

In [7]:
# oof_age.to_csv("b/oof_age_st_lvl2.csv", index=False)
# test_age.to_csv("b/test_age_st_lvl2.csv", index=False)
target = pd.read_parquet("public_train.pqt")
oof_age = pd.read_csv("cold_age/oof_cold_age_l2_lgb_automl.csv")
test_age = pd.read_csv("cold_age/test_cold_age_l2_lgb_automl.csv")
oof_age.shape

(270000, 13)

In [8]:
oof_age = oof_age.merge(target, on='user_id')
oof_age.shape

(270000, 15)

In [9]:
def age_bucket(x):
    return bisect.bisect_left([0,25,35,45,55,65], x)

oof_age['age'] = oof_age['age'].map(age_bucket)

In [10]:
# rnn_oof = pd.read_csv("warm_age/age_lstm_oof_1024.csv")
# print(rnn_oof.shape)
# rnn_test = pd.read_csv("warm_age/age_lstm_test_1024.csv")
# rnn_oof.head()

In [11]:
# oof_age = oof_age.merge(rnn_oof, on='user_id')
# test_age = test_age.merge(rnn_test, on='user_id')
# print(oof_age.shape)

In [12]:
# train = train[~train['is_male'].isna() & (train['is_male'].apply(lambda x: x != 'NA'))]
# train['is_male'] = train['is_male'].astype('int')
# print(len(train))

In [13]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
import time


roles = {'target': 'age',
                 'drop': ['user_id', 'index', "is_male"]}
default_nn_params = {
    "n_epochs": 200,
}
default_nn_params_2 = {
    "n_epochs": 100,
}
automl = TabularAutoML(
    task = Task(name='multiclass', metric="f1_weighted"),
    cpu_limit = 24,
    timeout=3600*1500,
    gpu_ids='0',
    debug=True,
    general_params = {"use_algos": [["linear_l2"]]},
    nn_pipeline_params = {"use_te": True,  "max_intersection_depth": 0},
    reader_params = {'cv': 10, 'random_state': 43},
)
start_time = time.time()
oof_pred = automl.fit_predict(
        oof_age,
        roles=roles,
        verbose = 1
    )
end_time = time.time()

[19:38:07] Stdout logging level is INFO.
[19:38:07] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[19:38:07] Task: multiclass

[19:38:07] Start automl preset with listed constraints:
[19:38:07] - time: 5400000.00 seconds
[19:38:07] - CPU: 24 cores
[19:38:07] - memory: 16 GB

[19:38:07] Train data shape: (270000, 15)

[19:38:09] Layer 1 train process start. Time left 5399997.39 secs
[19:38:09] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[19:40:02] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.3106935367070877
[19:40:02] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[19:40:02] Time left 5399884.64 secs

[19:40:02] Layer 1 training completed.

[19:40:02] Automl preset training completed in 115.36 seconds

[19:40:02] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (10 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) 



In [ ]:
# 0.4942498264611808

In [14]:
print(f"spend time: {end_time - start_time}")
print(automl.create_model_str_desc())

spend time: 115.38690447807312
Final prediction for new objects (level 0) = 
	 1.00000 * (10 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) 


In [15]:
test_age.sort_values("user_id", inplace=True)
# class_mapping = {v:k for k, v in automl.outer_pipes[0].ml_algos[0].models[0][0].reader.class_mapping.items()}
test_pred_age_probs = automl.predict(test_age).data
# test_pred_age_probs = np.hstack([test_pred_age_probs[:, class_mapping[i]-1].reshape(-1, 1) for i in range(6)])
with open('stack_lvl2_age_cold_final.npy', 'wb') as f:
    np.save(f, test_pred_age_probs)

In [16]:
list(np.argmax(test_pred_age_probs, axis=1))

[1,
 1,
 3,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 4,
 4,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 3,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 3,
 3,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 0,
 3,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
